In [2]:
import numpy as np
import cv2
import random
import os, subprocess
import sys
sys.path.append('C:\\Program Files\\\Python36\\Scripts')
import gdal_merge as gm
import glob
import itertools
import math
import re
from osgeo import gdal, gdal_array, ogr, osr
from matplotlib import pyplot as plt
from PIL import Image as pil_image
from keras.preprocessing.image import img_to_array
from sklearn.metrics import confusion_matrix
from pandas_ml import ConfusionMatrix
import itertools
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [2]:
# using:
#gdal_merge.py [-o out_filename] [-of out_format] [-co NAME=VALUE]*
#              [-ps pixelsize_x pixelsize_y] [-tap] [-separate] [-q] [-v] [-pct]
#              [-ul_lr ulx uly lrx lry] [-init "value [value...]"]
#              [-n nodata_value] [-a_nodata output_nodata_value]
#              [-ot datatype] [-createonly] input_files
#             dtype="Float32"

In [6]:
# read radiance solar corrected geoTiff files to create mosaic per each island/each day/folder
#----------------------------------------------------------------------------------------------

main_dir ='C:\\NeMO-Net Data\\Fiji Raw Imagery\\'
islands  = ['Fulaga', 'Kobara', 'Mago','Matuka','Moala','Nayau', 'Totoya','Tuvuca', 'Vanua Balavu','Vanua Vatu']
#islands  = ['Cicia']
islands  = ['Vanua Balavu']

for i in islands:
    print(i)
    island_dir = main_dir + i + "/"
    # search for subsir in each island
    subdir_ = [os.path.join(island_dir, o) for o in os.listdir(island_dir) 
                    if os.path.isdir(os.path.join(island_dir,o))]
    print(subdir_)
    
    for f in subdir_:
        dir_str = f + "/" + "*MUL"
        subdir = glob.glob(dir_str) 
        print(subdir)
        
        # mosaic processed tif files in each directory in subdir:
        for ff in subdir:
            print("subdir: ",ff)
            ## load geotiff files to mosaic:
            tiff_str = ff + "/" + "*rad_solar_corrected.tif"
            tif_files   = glob.glob(tiff_str)
            gmc = os.path.join('C:\\','Program Files','Python36', 'Scripts','gdal_merge.py')
            out = ff + '/mosaic_rad_solar_corrected_all.tif'
            print("out_path:", out)
            merge_command = ["python",gmc, "-o", out]
            [merge_command.append(s) for s in tif_files]
            
            subprocess.call(merge_command,shell=True)
                
            # upload mosaiced file for plotting
                
            mf = ff + "/" + "mosaic_rad_solar_corrected_all.tif"
            print("uploading: ", mf, " to plot")
            img = gdal.Open(mf)
            xsize = img.RasterXSize
            ysize = img.RasterYSize
            projection = img.GetProjection()
            image = np.zeros((ysize,xsize,img.RasterCount))

            for band in range(img.RasterCount):
                band += 1
                imgband = img.GetRasterBand(band)
                image[:,:,band-1]  = imgband.ReadAsArray()
                
            print("mosaiced tif shape: ", image.shape)            
            # plot mosaic as RGB
            img_RGB = np.rollaxis(np.asarray([1/200*image[:,:,4],\
                                                      1/300*image[:,:,2], \
                                                      1/300*image[:,:,1]]),0,3)

            plt.figure(1)
            plt.imshow(img_RGB)
            plt.title('mosaiced tiff')
            #plt.show()

                            
            ## save plot as .png
            plotname = ff + "/" + "mosaiced_all_rad_solar_corrected.png"
            print("saving mosaiced tiff into: ", plotname)
            plt.savefig(plotname, bbox_inches='tight',dpi=1000)


Vanua Balavu
['C:\\NeMO-Net Data\\Fiji Raw Imagery\\Vanua Balavu/052985603100_01', 'C:\\NeMO-Net Data\\Fiji Raw Imagery\\Vanua Balavu/054850703010_01']
['C:\\NeMO-Net Data\\Fiji Raw Imagery\\Vanua Balavu/052985603100_01\\052985603100_01_P001_MUL', 'C:\\NeMO-Net Data\\Fiji Raw Imagery\\Vanua Balavu/052985603100_01\\052985603100_01_P002_MUL', 'C:\\NeMO-Net Data\\Fiji Raw Imagery\\Vanua Balavu/052985603100_01\\052985603100_01_P003_MUL', 'C:\\NeMO-Net Data\\Fiji Raw Imagery\\Vanua Balavu/052985603100_01\\052985603100_01_P004_MUL', 'C:\\NeMO-Net Data\\Fiji Raw Imagery\\Vanua Balavu/052985603100_01\\052985603100_01_P005_MUL', 'C:\\NeMO-Net Data\\Fiji Raw Imagery\\Vanua Balavu/052985603100_01\\052985603100_01_P006_MUL']
subdir:  C:\NeMO-Net Data\Fiji Raw Imagery\Vanua Balavu/052985603100_01\052985603100_01_P001_MUL
out_path: C:\NeMO-Net Data\Fiji Raw Imagery\Vanua Balavu/052985603100_01\052985603100_01_P001_MUL/mosaic_rad_solar_corrected_all.tif
uploading:  C:\NeMO-Net Data\Fiji Raw Imagery\V

In [ ]:
# upload mosaiced multi-band gtiff and save only RGB as tiff
#------------------------------------------------------------

main_dir ='C:\\NeMO-Net Data\\Fiji Raw Imagery\\'
islands  = ['Fulaga', 'Kobara', 'Mago','Matuka','Moala','Nayau', 'Totoya','Tuvuca', 'Vanua Balavu','Vanua Vatu']
#islands  = ['Cicia']

for i in islands:
    print(i)
    island_dir = main_dir + i + "/"
    # search for subsir in each island
    subdir_ = [os.path.join(island_dir, o) for o in os.listdir(island_dir) 
                    if os.path.isdir(os.path.join(island_dir,o))]
    print(subdir_)
    
    for f in subdir_:
        dir_str = f + "/" + "*MUL"
        subdir = glob.glob(dir_str) 
        print(subdir)
        
        # mosaic processed tif files in each directory in subdir:
        for ff in subdir:
            print("subdir: ",ff)
            ## load geotiff files to mosaic:
            tiff_str = ff + "/" + "*rad_solar_corrected.tif"
            tif_files   = glob.glob(tiff_str)
            gmc = os.path.join('C:\\','Program Files','Python36', 'Scripts','gdal_merge.py')
            out = ff + '/mosaic_rad_solar_corrected_all.tif'
            print("out_path:", out)
            merge_command = ["python",gmc, "-o", out]
            [merge_command.append(s) for s in tif_files]
            
            subprocess.call(merge_command,shell=True)
                
            # upload mosaiced file for plotting
                
            mf = ff + "/" + "mosaic_rad_solar_corrected_all.tif"
            print("uploading: ", mf, " to plot")
            img = gdal.Open(mf)
            xsize = img.RasterXSize
            ysize = img.RasterYSize
            projection = img.GetProjection()
            image = np.zeros((ysize,xsize,img.RasterCount))

            for band in range(img.RasterCount):
                band += 1
                imgband = img.GetRasterBand(band)
                image[:,:,band-1]  = imgband.ReadAsArray()
                
            print("mosaiced tif shape: ", image.shape)            
            # plot mosaic as RGB
            img_RGB = np.rollaxis(np.asarray([1/200*image[:,:,4],\
                                                      1/300*image[:,:,2], \
                                                      1/300*image[:,:,1]]),0,3)

            plt.figure(1)
            plt.imshow(img_RGB)
            plt.title('mosaiced tiff')
            #plt.show()

                            
            ## save plot as .png
            plotname = ff + "/" + "mosaiced_all_rad_solar_corrected.png"
            print("saving mosaiced tiff into: ", plotname)
            plt.savefig(plotname, bbox_inches='tight',dpi=1000)
